# EPF Toolbox: forecasting models demo

Second notebook to generate day-ahead forecasts with `epftoolbox.models`.
- Mirrors the examples in `examples/` and the [models documentation](https://epftoolbox.readthedocs.io/en/latest/modules/models.html).
- Reuses data cached in `datasets/` from `01_load_data.ipynb`.
- Saves forecasts to `forecasts_local/` (created below).


In [1]:
from pathlib import Path

repo_root = Path.cwd().resolve()
if repo_root.name == 'Notebooks':
    repo_root = repo_root.parent

data_path = repo_root / 'datasets'
work_path = repo_root / 'forecasts_local'
work_path.mkdir(exist_ok=True)

print(f'Repo root: {repo_root}')
print(f'Data path: {data_path}')
print(f'Experiments path: {work_path}')


Repo root: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox
Data path: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/datasets
Experiments path: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/forecasts_local


# Verify GPU compute
In Temrinal, run: 
```bash
watch -n0.2 nvidia-smi  
```

In [2]:
import sys, os

print("sys.executable:", sys.executable)
print("sys.prefix:", sys.prefix)
print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))

sys.executable: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv/bin/python3
sys.prefix: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv
VIRTUAL_ENV: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv


In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices("GPU"))
with tf.device("/GPU:0"):
     a = tf.random.normal([2000, 2000])
     b = tf.random.normal([2000, 2000])
     _ = tf.matmul(a, b)  # warms up JIT

2025-12-01 16:35:46.820872: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 16:35:46.844965: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-01 16:35:46.844987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-01 16:35:46.845640: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-01 16:35:46.849904: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-12-01 16:35:47.865856: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-01 16:35:47.889779: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-01 16:35:47.891839: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
import tensorflow as tf, time

tf.debugging.set_log_device_placement(True)  # optional, see placements

with tf.device("/GPU:0"):
    a = tf.random.normal([8192, 8192])
    b = tf.random.normal([8192, 8192])
    for i in range(3):
        t0 = time.time()
        c = tf.matmul(a, b)
        _ = c.numpy()  # force materialization
        print(f"iter {i} took {time.time() - t0:.3f}s")

iter 0 took 0.191s
iter 1 took 0.187s
iter 2 took 0.186s


## LEAR daily recalibration (CPU extensive)

Quick run of the LEAR example (`epftoolbox.models.evaluate_lear_in_test_dataset`) to produce a forecast CSV in `forecasts_local/`.


In [ ]:
%%time
# 6 hours of run time
from epftoolbox.models import evaluate_lear_in_test_dataset

dataset = 'PJM'
years_test = 2
calibration_window = 364 * 3

forecast_lear = evaluate_lear_in_test_dataset(
    path_datasets_folder=str(data_path),
    path_recalibration_folder=str(work_path),
    dataset=dataset,
    years_test=years_test,
    calibration_window=calibration_window,
)

forecast_lear.head()


/home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv/lib/python3.10/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


2016-12-27 - sMAPE: 5.99%  |  MAE: 1.604
2016-12-28 - sMAPE: 4.67%  |  MAE: 1.298
2016-12-29 - sMAPE: 4.51%  |  MAE: 1.274
2016-12-30 - sMAPE: 5.07%  |  MAE: 1.451
2016-12-31 - sMAPE: 6.30%  |  MAE: 1.682
2017-01-01 - sMAPE: 8.03%  |  MAE: 2.036
2017-01-02 - sMAPE: 7.94%  |  MAE: 2.016
2017-01-03 - sMAPE: 7.90%  |  MAE: 2.009
2017-01-04 - sMAPE: 7.64%  |  MAE: 1.971
2017-01-05 - sMAPE: 7.65%  |  MAE: 2.063
2017-01-06 - sMAPE: 7.37%  |  MAE: 2.030
2017-01-07 - sMAPE: 7.33%  |  MAE: 2.063
2017-01-08 - sMAPE: 8.02%  |  MAE: 2.352
2017-01-09 - sMAPE: 8.89%  |  MAE: 2.639
2017-01-10 - sMAPE: 8.96%  |  MAE: 2.629
2017-01-11 - sMAPE: 9.45%  |  MAE: 2.693
2017-01-12 - sMAPE: 9.29%  |  MAE: 2.630
2017-01-13 - sMAPE: 9.07%  |  MAE: 2.576
2017-01-14 - sMAPE: 8.96%  |  MAE: 2.549
2017-01-15 - sMAPE: 8.95%  |  MAE: 2.537
2017-01-16 - sMAPE: 8.73%  |  MAE: 2.482
2017-01-17 - sMAPE: 8.82%  |  MAE: 2.504
2017-01-18 - sMAPE: 8.72%  |  MAE: 2.475
2017-01-19 - sMAPE: 8.70%  |  MAE: 2.479
2017-01-20 - sMA

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-27,19.100844,18.264595,17.53075,17.434241,18.22411,19.853216,23.788848,25.228666,26.251533,27.419878,...,26.20889,26.338958,28.022928,35.886231,32.565618,31.739757,31.986766,28.902147,24.976016,22.516413
2016-12-28,21.947657,21.407247,20.915056,20.843612,21.378066,23.707062,30.277734,33.646398,31.340376,30.944352,...,25.61739,26.071015,27.769258,35.816337,36.297189,34.476443,32.25681,29.718951,25.991414,24.034067
2016-12-29,21.963601,20.970229,20.101968,20.117763,20.486912,22.704496,28.13348,32.937301,30.116596,31.026365,...,28.609186,28.888347,30.637242,38.876011,36.60512,35.55204,35.255296,30.515633,26.571027,24.230899
2016-12-30,22.895867,21.905473,21.212615,21.177958,21.596962,23.675649,28.202682,32.008085,31.970438,33.674966,...,29.523996,29.002559,31.157213,39.438211,36.56299,35.710511,36.192968,32.226899,28.493623,25.891891
2016-12-31,23.020518,21.974983,21.226,20.678976,20.496707,21.093947,22.753666,24.696514,26.168603,26.245395,...,17.416643,17.177852,18.647874,28.98324,24.530161,20.954127,22.603399,23.652396,22.838756,21.804186


## DNN daily recalibration (requires hyperparameters)

The DNN helpers mirror `examples/recalibrating_dnn_simplified.py`, but they need a trials file from `examples/optimizing_hyperparameters_dnn.py`.
The cell below checks for the expected file in `forecasts_local/` and only runs when it exists.


### Optional: create DNN hyperparameter trials

The DNN helper needs a trials file (hyperparameter search). If it is missing, run a small hyperopt sweep here to generate it. Increase `max_evals` for a proper search; the default keeps this demo quick.


In [ ]:
# 8 min run time
from epftoolbox.models import hyperparameter_optimizer

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 4
years_test = 2
dataset = 'PJM'
experiment_id = 1
begin_test_date = None
end_test_date = None
max_evals = 10  # increase for a full search; small for a quick demo

trials_file_name = (
    f'DNN_hyperparameters_nl{nlayers}_dat{dataset}_YT{years_test}'
    f"{'_SF' if shuffle_train else ''}"
    f"{'_DA' if data_augmentation else ''}"
    f'_CW{calibration_window}_{experiment_id}'
)
trials_path = work_path / trials_file_name

if trials_path.exists():
    print('Trials file already exists, skipping hyperparameter search:')
    print(trials_path)
else:
    print('Running a small hyperparameter search to create trials file...')
    hyperparameter_optimizer(
        path_datasets_folder=str(data_path),
        path_hyperparameters_folder=str(work_path),
        new_hyperopt=True,
        max_evals=max_evals,
        nlayers=nlayers,
        dataset=dataset,
        years_test=years_test,
        calibration_window=calibration_window,
        shuffle_train=bool(shuffle_train),
        data_augmentation=bool(data_augmentation),
        experiment_id=experiment_id,
        begin_test_date=begin_test_date,
        end_test_date=end_test_date,
    )
    print('Saved trials to:', trials_path)


Running a small hyperparameter search to create trials file...


2025-12-01 22:36:01.712813: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-12-01 22:36:03.226730: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f691d8f79b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-01 22:36:03.226753: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-12-01 22:36:03.230440: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-01 22:36:03.246060: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1764599763.310637 3192343 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.




Tested 1/10 iterations.
Best MAE - Validation Dataset
  MAE: 6.5 | sMAPE: 15.01 %

Best MAE - Test Dataset
  MAE: 4.3 | sMAPE: 16.02 %




Tested 2/10 iterations.
Best MAE - Validation Dataset
  MAE: 4.7 | sMAPE: 11.92 %

Best MAE - Test Dataset
  MAE: 3.7 | sMAPE: 14.40 %




Tested 3/10 iterations.
Best MAE - Validation Dataset
  MAE: 4.7 | sMAPE: 11.92 %

Best MAE - Test Dataset
  MAE: 3.7 | sMAPE: 14.40 %


...



Tested 6/10 iterations.
Best MAE - Validation Dataset
  MAE: 4.7 | sMAPE: 11.92 %

Best MAE - Test Dataset
  MAE: 3.7 | sMAPE: 14.40 %




Tested 7/10 iterations.
Best MAE - Validation Dataset
  MAE: 4.7 | sMAPE: 11.92 %

Best MAE - Test Dataset
  MAE: 3.7 | sMAPE: 14.40 %




Tested 8/10 iterations.
Best MAE - Validation Dataset
  MAE: 4.7 | sMAPE: 11.92 %

Best MAE - Test Dataset
  MAE: 3.7 | sMAPE: 14.40 %




Tested 9/10 iterations.
Best MAE - Validation Dataset
  MAE: 4.7 | sMAPE: 11.92 %

Best MAE - Test Dataset
  MAE: 3.7 | sMAPE: 14.40 %


Saved trials to: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/forecasts_local/DNN_hyperparameters_nl2_datPJM_YT2_SF_CW4_1


In [ ]:
%%time
# 6 hours of run time
from epftoolbox.models import evaluate_dnn_in_test_dataset

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 4
years_test = 2
dataset = 'PJM'
experiment_id = 1
begin_test_date = None
end_test_date = None

trials_file_name = (
    f'DNN_hyperparameters_nl{nlayers}_dat{dataset}_YT{years_test}'
    f"{'_SF' if shuffle_train else ''}"
    f"{'_DA' if data_augmentation else ''}"
    f'_CW{calibration_window}_{experiment_id}'
)
trials_path = work_path / trials_file_name

if trials_path.exists():
    forecast_dnn = evaluate_dnn_in_test_dataset(
        experiment_id=experiment_id,
        path_hyperparameter_folder=str(work_path),
        path_datasets_folder=str(data_path),
        path_recalibration_folder=str(work_path),
        nlayers=nlayers,
        dataset=dataset,
        years_test=years_test,
        shuffle_train=bool(shuffle_train),
        data_augmentation=bool(data_augmentation),
        calibration_window=calibration_window,
        new_recalibration=True,
        begin_test_date=begin_test_date,
        end_test_date=end_test_date,
    )
    display(forecast_dnn.head())
else:
    print('Skipping DNN run – hyperparameter trials file not found:')
    print(trials_path)
    print('Run examples/optimizing_hyperparameters_dnn.py to generate it.')

2016-12-27 - sMAPE: 3.30%  |  MAE: 0.805


2016-12-28 - sMAPE: 2.89%  |  MAE: 0.775


2016-12-29 - sMAPE: 3.65%  |  MAE: 1.012


2016-12-30 - sMAPE: 3.32%  |  MAE: 0.939


2016-12-31 - sMAPE: 4.10%  |  MAE: 1.149


...

2018-12-22 - sMAPE: 13.64%  |  MAE: 3.607


2018-12-23 - sMAPE: 13.62%  |  MAE: 3.603


2018-12-24 - sMAPE: 13.62%  |  MAE: 3.602


,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-27,19.722332,18.762646,17.875179,17.425495,17.95731,19.831326,23.064501,25.105787,25.819361,26.450245,...,25.16168,24.853588,27.04982,32.59742,31.401173,29.773619,28.484137,26.288269,23.462406,21.424294
2016-12-28,21.940588,21.12418,20.55291,20.366854,21.05826,22.998348,28.64551,31.444799,30.458847,30.310175,...,26.189642,25.870569,27.597801,34.093395,34.890057,33.804352,31.937109,28.749397,25.460709,23.513735
2016-12-29,22.071405,21.205957,20.397139,20.083685,20.608782,22.427935,28.268677,31.916702,30.455582,30.337023,...,25.334558,24.985592,26.192137,32.253166,33.739529,31.842726,29.774796,27.318352,24.706926,22.724243
2016-12-30,22.564846,21.477358,20.524448,20.290924,20.599266,22.256643,27.309494,30.56542,30.054205,30.435417,...,26.660389,26.542553,27.996674,35.454453,34.793991,32.478378,31.775034,29.179237,26.096451,23.821146
2016-12-31,23.019995,21.957537,21.048809,20.616011,20.796965,22.124723,25.520527,28.397703,29.058786,29.583309,...,25.807159,25.835478,27.768059,36.155087,34.556076,31.743786,30.481173,28.309082,25.812309,23.724516


CPU times: user 5h 50min 53s, sys: 45min 42s, total: 6h 36min 36s
Wall time: 6h 4min 10s
